In [347]:
#Preprocessing 1 : Data Load

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import metrics

import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import *
import category_encoders as ce
from sklearn.model_selection import GridSearchCV, KFold
import graphviz

from sklearn import metrics

from sklearn.svm import SVC
from sklearn import tree
from sklearn.linear_model import LogisticRegression as LogicRegression

import category_encoders as ce
import pandas as pd
import numpy as np
import tensorflow as tf

pd.set_option('display.max_row', 10)
pd.set_option('display.max_columns', 10)

df = pd.read_csv("breast-cancer-wisconsin.data")
df.columns = ['Sample code number', 'clump thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class']
df




,Sample code number,clump thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,...,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1002945,5,4,4,5,...,10,3,2,1,2
1,1015425,3,1,1,1,...,2,3,1,1,2
2,1016277,6,8,8,1,...,4,3,7,1,2
3,1017023,4,1,1,3,...,1,3,1,1,2
4,1017122,8,10,10,8,...,10,9,7,1,4
...,...,...,...,...,...,...,...,...,...,...,...
693,776715,3,1,1,1,...,2,1,1,1,2
694,841769,2,1,1,1,...,1,1,1,1,2
695,888820,5,10,10,3,...,3,8,10,2,4
696,897471,4,8,6,4,...,4,10,6,1,4


In [348]:
#Preprocessing 2 : 

#Drop row that has ? value
df = df[df.iloc[:,6] != '?']

#Drop unnecessary data
df = df.drop(["Sample code number"], axis=1)
df = df.drop(["Mitoses"], axis=1)

df_label = df.drop(["Class"], axis = 1)
df_target = df["Class"]

print(df_label)
print(df_target)

     clump thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0                  5                        4                         4   
1                  3                        1                         1   
2                  6                        8                         8   
3                  4                        1                         1   
4                  8                       10                        10   
..               ...                      ...                       ...   
693                3                        1                         1   
694                2                        1                         1   
695                5                       10                        10   
696                4                        8                         6   
697                4                        8                         8   

     Marginal Adhesion  Single Epithelial Cell Size Bare Nuclei  \
0                    5          

In [349]:
# Preprocessing 3 : 

df_label.isna().sum()

df_label.dropna(axis = 0, how = 'any', inplace = True)

df_label #682

df_label.columns

Index(['clump thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Marginal Adhesion',
       'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
       'Normal Nucleoli'],
      dtype='object')

In [350]:
#Encoding Function
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
    
def Encoding(dataframe, mode) :

    categorized_column = dataframe.select_dtypes(include=["category", "object"])
    
    if mode == "label" :
        res = categorized_column.apply(LabelEncoder().fit_transform)

    elif mode == "ordinal" :
        enc = ce.OrdinalEncoder()
        if len(categorized_column.columns) == 1 :
            res = enc.fit_transform(categorized_column)
        else :
            res = categorized_column.apply(enc.fit_transform)
        
    elif mode == "onehot" :
        enc = ce.OneHotEncoder()
        if len(categorized_column.columns) == 1 :
            res = enc.fit_transform(categorized_column)
        else :
            res = categorized_column.apply(enc.fit_transform)
            
    
        
    #Merge into original dataframe
    deleted_attr = dataframe.drop(categorized_column, axis = 1)
    dataframe = pd.concat([deleted_attr, res],axis=1, join='inner')
    
    
    return dataframe

In [351]:
#Scaling
def Scaling (dataframe, mode) :

    if mode == "minmax" :
        scaler = preprocessing.MinMaxScaler()
    
    elif mode == "standard" :
        scaler = preprocessing.StandardScaler()
    
    elif mode == "robust" :
        scaler = preprocessing.RobustScaler()
    
    elif mode == "normal" :
        scaler = Normalizer()
    
    scaled_dataframe = scaler.fit_transform(dataframe)
    
    scaled_dataframe = pd.DataFrame(data=scaled_dataframe, columns=dataframe.columns)

    return scaled_dataframe

In [352]:
#DecisionTree_entropy function
def DecisionTree_entropy(label, target) :

	model = tree.DecisionTreeClassifier(criterion="entropy")

	# 정규화된 데이터에 학습
	clf = model.fit(label, target)
	dot_data = tree.export_graphviz(clf, out_file = None, feature_names = df_label.columns, class_names = target.name, filled = True, rounded = True)
	graph = graphviz.Source(dot_data)
	
	return graph.view()
	

In [353]:
def DecisionTree_gini(label, target) :
    
	model = tree.DecisionTreeClassifier()

	# 정규화된 데이터에 학습
	clf = model.fit(label, target)
	dot_data = tree.export_graphviz(clf, out_file = None, feature_names = df_label.columns, class_names = target.name, filled = True, rounded = True)
	graph = graphviz.Source(dot_data)
	
	return graph.view()

In [354]:
#SVM function
def SVM(label, target):

	kfold = KFold(n_splits=5, shuffle=True, random_state=0)
	find_grid = SVC(kernel='rbf', random_state=100)
	parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100],
               'gamma': [0.001, 0.01, 0.1, 1, 10, 25, 50, 100]}

	gsSVM = GridSearchCV(find_grid,
                      param_grid=parameters, cv=kfold)

	gsSVM.fit(label, target)
	score = gsSVM.score(label, target)
	return print(gsSVM.best_params_, score)


In [355]:
#LogisticRegression function
def LogisticRegression(label, target):

    kfold = KFold(n_splits=5, shuffle=True, random_state=0)
    logisticRegression = LogicRegression()
    solver = {
            'solver':['lbfgs', 'newton-cg', 'liblinear', 'saga']
        }
    gsLogistic = GridSearchCV(logisticRegression, param_grid=solver, cv=kfold)

    gsLogistic.fit(label, target)
    return print(gsLogistic.best_params_, gsLogistic.best_score_)

    

In [356]:
def predict(model, label, target):
    if model == "DecisionTree_entropy":
        DecisionTree_entropy(label, target)
    elif model == "DecisionTree_gini":
        DecisionTree_gini(label, target)
    elif model == "SVM":
        SVM(label, target)
    elif model == "LogisticRegression":
        LogisticRegression(label, target)


In [357]:

def AutoML(scaler, encoder, model, label, target) :
    
    if not isinstance(train, pd.DataFrame):
        raise TypeError
    # if not isinstance(scaler, dict):
    #     raise TypeError
    
    predicted = {}

    print("Encoder Type : ",encoder)
    print("Scaler Type : ",scaler)
    print("Model Type : ",model)

    hyper_param = {
    "k-means": [5],
    "em": [5],
    "clarans": {"numCluster":[4], "numLocal":[5], "maxNeighbor":[5]},
    "dbscan": {"eps":[0.1], "minSample":[4]}
}

    for e in encoder:
        e_train = Encoding(train, e)
        print("------------------------------------")
        print("Encoder Type: ",e)
        for s in scaler:
            print("------------------------------------")
            print("Scaler Type : " , s)
            s_train = Scaling(e_train,s)
            for m in model:
                print("------------------------------------")
                print("Model Type : ",m)
                predicted = predict(m, label, target)
                


In [358]:
#Main

encoder =  []
scaler = []
model = []

train = df_label

print("------------------------------------")
print("Type Your Encoder Type with String (Ex. label ordinal)")
print("Encoder type : <label, oridnal, onehot>")
encoder = list(input().split())
print(encoder)

print("------------------------------------")
print("Type Your Scaler Type with String (Ex. minmax standard)")
print("Scaler type : <minmax, standard, robust>")
scaler = list(input().split())

print("------------------------------------")
print("Type Your Classification Model with String (Ex. DecisionTree_entropy DecisionTree_gini LogisticRegression SVM)")
print("Model type : <DecisionTree_entropy, DecisionTree_gini, LogisticRegression, SVM>")
model = list(input().split())

AutoML(scaler, encoder, model, df_label, df_target)


------------------------------------
Type Your Encoder Type with String (Ex. label ordinal)
Encoder type : <label, oridnal, onehot>
['label']
------------------------------------
Type Your Scaler Type with String (Ex. minmax standard)
Scaler type : <minmax, standard, robust>
------------------------------------
Type Your Clustering Model Type with String (Ex. k-means em clarans dbscan affinity)
Model type : <Decision_tree, SVM, LogisticRegression, dbscan, affinity>
Encoder Type :  ['label']
Scaler Type :  ['minmax']
Model Type :  ['SVM']
------------------------------------
Encoder Type:  label
------------------------------------
Scaler Type :  minmax
------------------------------------
Model Type :  SVM
{'C': 50, 'gamma': 0.001} 0.9721407624633431
